# Bioinformatics Platform using JupyterLab

Author: Robert Bradford

This is a thesis project using JupyterLab to facilitate the study of BIOC-4010 course. This tutorial implements the following basic modules using Biopython, NGLView。
1. Databases and sequencing file formats
2. Translating DNA sequences into a protein sequence
3. Calculating DNA GC content
4. Pairwise sequence alignments using Needleman and Waterman algorithms
5. Substitution matrices
6. BLAST
7. Display a 3D structure

Requirements:

To run this notebook successfully, it is recommended to use Miniconda + JupyterLab and install the required packages and extensions. The notebook shall also work on [Google Colab](https://colab.research.google.com/) or [Binder](https://jupyter.org/binder) but this has not been tested.

The following packages are required and can be installed using conda. It is recommended to create a new environment and install these packages. You can use the [nglview-jupyterlab.sh script](https://github.com/nglviewer/nglview/blob/master/devtools/nglview-jupyterlab.sh) to install the nglview related packages.
* python 3.8+
* jupyterlab 2.1+
* biopython 1.7+
* ipywidgets 7.5+
* nodejs 12.0.0+, required for the jupyter-labextensions
* nglview 2.7+
    * if you do not use the nglview-jupyterlab.sh script, run the following two commands manually after you install jupyterlab
    * `jupyter-labextension install @jupyter-widgets/jupyterlab-manager`
    * `jupyter-labextension install nglview-js-widgets@$nglviewversion` where `$nglviewversion` is the version of the `nglview` installed package, which can be inspected with `conda list nglview`.
    

This notebook has been test on:
1. miniconda3
2. command line Windows (NGLView generated 3D structures do not load)
3. 

## Setup environment

In [1]:
import Bio
from Bio.Seq import Seq
from Bio import SeqIO
from Bio.Data import CodonTable
import ipywidgets as widgets
from Bio.PDB import *
import os
import sys
import nglview as nv
import platform
print("Python version",sys.version_info)
print("Biopython version", Bio.__version__)

Python version sys.version_info(major=3, minor=9, micro=1, releaselevel='final', serial=0)
Biopython version 1.77


## Chapter 1 Databases and File Formats
The use of bioinformatics typically involves dealing with vast amounts of biological data. This can range from DNA sequences, protein sequences, protein structures, and all their associated annotations. This information is stored into various file formats and can be manipulated and read using biopython. Each file format has unique characteristics and information that a biologist might want to manipulate. In the interest of being able to access and manipulate this data, one must be able to understand these file formats, and where to access this information from. 

### Databases
Biological data, ranging from gene sequences to protein sequences is stored in databases. 

Beginning with DNA, there are **three** main databases commonly used when dealing with nucleotide sequences: 

1. GenBank, provided by the National Center for Biotechnology Information (NCBI)
2. European Molecular Biology Laboratory (EMBL)-Bank, provided by the European Bioinformatics Institute (EBI)
3. DNA Database of Japan (DDBJ), provided by the National Institute of Genetics in Mishima

These databases are coordinated by the International Nucleotide Sequence Database Collaboration 
(INSDC). Each of these databases are but a part of the resources and other database of interest provided by the NCBI, EBI, and DDBJ.

### File Formats
As mentioned, databases store a wide range of relevent biological information. Some of this information may include large DNA sequences, mRNA products, or proteins sequences. These sequences alone are fairly simple linear arangements of nucleotides and amino acids. However, the context around these sequences, such as the name of the organism, chromosome, gene, or intron they are taken from is also important. 

In order to keep track of this information such that its kept organized and a biologist can understand it, several standardized file formats have been developed, a list of which can be found [here](https://www.algosome.com/articles/bioinformatics-sequence-file-formats.html). Some of these include:

* GenBank, which can store a wide variety of sequence information
* EMBL, similar to GenBank, but refit for the EBI database
* ABI, which stores pure sequence information in binary. usually only used in special cases.
* PDB, protein database formate, used to store sequence information, and the protein's 3D structure gained through crystalography
* MLD, mostly stores information regarding smaller molecular strucutres, very similar to PDB
* BAM/SAM, stores next generation sequencing data, containing both a binary sequence readout and an alphabetical sequence readout.
* SFF, also stores next generation sequencing information, specifically the sequencing information from Ion-Torrent and Roche's '454'.

Each database uses their own unique type of format, however they each have similarities between them. This document covers some of the more commonly used file types typically encountered by bioinformatics students.

#### FASTA File Format
FASTA is one of the simplist of the sequence data formats. FASTA usually contains some idintifying information for the sequence of interest in a header, followed either by the actual DNA or protein sequence. An official description of the file format can be found [here](https://blast.ncbi.nlm.nih.gov/Blast.cgi?CMD=Web&PAGE_TYPE=BlastDocs&DOC_TYPE=BlastHelp).

##### FASTA files look something like this:

Fasta files are quite simple to read:
* Generally, Fasta files begin with an information line, usually denoted with an ">" symbole.
* Stored sequence pertaining to that information line follows.


#### GenBank File Format
GenBank, EMBL, and DDBJ offer a wide range of molecular sequence data. We will use GenBank to demonstrate how this sequence information is organized and how to access it.GenBank mainly deals with genomic DNA sequences, their theoretically transcribed pre-mRNA, mRNA products, and their translated protein sequences. GenBank provides this and more information regarding specific proteins sequences through the GenBank file format.

GenBank uses a standardized file format to contain pertinent information for any given DNA sequence. The GenBank format helps keep information pertaining to a particular gene or protein relatively standard across platforms and help with ease of access. An offical description of each element in the format can be found [here](https://www.ncbi.nlm.nih.gov/Sitemap/samplerecord.html) at the NCBI's website.

##### GenBank files look something like this:

GenBank files appear vastly more complicated than FASTA files at an initial glance. However, FASTA files and GenBank files function quite similarly:
* instead of a simple identification line denoted by the ">" like in a FASTA file, Gene bank files can contain a fast amount of intial information pertaining to the gene of interest at the start of the file.
* GenBank files can contain many features related to specific sequences within the file.
* like FASTA files, GenBank files also contain the sequence information after the initial header.

#### Using data from from FASTA and GenBank files

Both GenBank and FASTA files store raw sequence data as well as some identifiers specific to those sequences. 
Biopython allows users to make novel FASTA and GenBank files with sequence data they'd like to store, and also allows users to pull data from existing files.
This chapter will use the example fasta file from the biopython tutorial [here](https://raw.githubusercontent.com/biopython/biopython/master/Doc/examples/ls_orchid.fasta) into any text file program as "ls_orchid.fasta" to show how to retrieve data from such files. Download the file, and put it into the same location as that of the bioinformatics tutorial file.

Using biopython, one can then extract sequence data from that file:

In [4]:
# This script uses "parse" to select the first entry in the FASTA file denoted by ">" 
# and stores that information into "record"
for record in SeqIO.parse("ls_orchid.fasta", "fasta"):
    
    # then prints the record ID, the seqence in that record, and the length of the sequence
    print(record.id)
    print(repr(record.seq))
    print(len(record))

gi|2765658|emb|Z78533.1|CIZ78533
Seq('CGTAACAAGGTTTCCGTAGGTGAACCTGCGGAAGGATCATTGATGAGACCGTGG...CGC', SingleLetterAlphabet())
740
gi|2765657|emb|Z78532.1|CCZ78532
Seq('CGTAACAAGGTTTCCGTAGGTGAACCTGCGGAAGGATCATTGTTGAGACAACAG...GGC', SingleLetterAlphabet())
753
gi|2765656|emb|Z78531.1|CFZ78531
Seq('CGTAACAAGGTTTCCGTAGGTGAACCTGCGGAAGGATCATTGTTGAGACAGCAG...TAA', SingleLetterAlphabet())
748
gi|2765655|emb|Z78530.1|CMZ78530
Seq('CGTAACAAGGTTTCCGTAGGTGAACCTGCGGAAGGATCATTGTTGAAACAACAT...CAT', SingleLetterAlphabet())
744
gi|2765654|emb|Z78529.1|CLZ78529
Seq('ACGGCGAGCTGCCGAAGGACATTGTTGAGACAGCAGAATATACGATTGAGTGAA...AAA', SingleLetterAlphabet())
733
gi|2765652|emb|Z78527.1|CYZ78527
Seq('CGTAACAAGGTTTCCGTAGGTGAACCTGCGGAAGGATCATTGTTGAGACAGTAG...CCC', SingleLetterAlphabet())
718
gi|2765651|emb|Z78526.1|CGZ78526
Seq('CGTAACAAGGTTTCCGTAGGTGAACCTGCGGAAGGATCATTGTTGAGACAGTAG...TGT', SingleLetterAlphabet())
730
gi|2765650|emb|Z78525.1|CAZ78525
Seq('TGTTGAGATAGCAGAATATACATCGAGTGAATCCGGAGGACCTGTGGTTATTCG...GC

**As you can see above**, there are 94 individual records in the example FASTA file, and each displays their ID, sequence and length.

Using biopython, we can also do the same thing with **GenBank files**, even with their extra layer of apparent complication compared to FASTA files. Simply download the example GenBank file availible in the biopython tutorial [here](https://raw.githubusercontent.com/biopython/biopython/master/Doc/examples/ls_orchid.gbk) into a text file as "ls_orchid.gbk". Make sure to put it into the same location as that of this bioinformatics platform file.

Using similar code to that used above for the 

In [7]:
# This script uses "parse" to select the first entry in the GenBank file, in this case the "LOCUS" line
# and stores that information into "record"
# (note with the file name you may have to add a .txt if using notepad, might not recognize the genbank file type)
for record in SeqIO.parse("ls_orchid.gbk.txt", "genbank"):
    
    # then prints the record ID, the seqence in that record, and the length of the sequence
    print(record.id)
    print(repr(record.seq))
    print(len(record))

Z78533.1
Seq('CGTAACAAGGTTTCCGTAGGTGAACCTGCGGAAGGATCATTGATGAGACCGTGG...CGC', IUPACAmbiguousDNA())
740
Z78532.1
Seq('CGTAACAAGGTTTCCGTAGGTGAACCTGCGGAAGGATCATTGTTGAGACAACAG...GGC', IUPACAmbiguousDNA())
753
Z78531.1
Seq('CGTAACAAGGTTTCCGTAGGTGAACCTGCGGAAGGATCATTGTTGAGACAGCAG...TAA', IUPACAmbiguousDNA())
748
Z78530.1
Seq('CGTAACAAGGTTTCCGTAGGTGAACCTGCGGAAGGATCATTGTTGAAACAACAT...CAT', IUPACAmbiguousDNA())
744
Z78529.1
Seq('ACGGCGAGCTGCCGAAGGACATTGTTGAGACAGCAGAATATACGATTGAGTGAA...AAA', IUPACAmbiguousDNA())
733
Z78527.1
Seq('CGTAACAAGGTTTCCGTAGGTGAACCTGCGGAAGGATCATTGTTGAGACAGTAG...CCC', IUPACAmbiguousDNA())
718
Z78526.1
Seq('CGTAACAAGGTTTCCGTAGGTGAACCTGCGGAAGGATCATTGTTGAGACAGTAG...TGT', IUPACAmbiguousDNA())
730
Z78525.1
Seq('TGTTGAGATAGCAGAATATACATCGAGTGAATCCGGAGGACCTGTGGTTATTCG...GCA', IUPACAmbiguousDNA())
704
Z78524.1
Seq('CGTAACAAGGTTTCCGTAGGTGAACCTGCGGAAGGATCATTGTTGAGATAGTAG...AGC', IUPACAmbiguousDNA())
740
Z78523.1
Seq('CGTAACCAGGTTTCCGTAGGTGAACCTGCGGCAGGATCATTGTTGAGACAGCAG...AAG', IUPAC

### Conclusion for databases and file formats

Overall, bioinformatics data is stored in large publically availible databases. These databases contain thousands of files that hold valueble biological data, and each database may specialize in different types of data and information. Each of these databases deals in standardized file formats that can be accessed remotely through bioinformatics software and read so as to be of use for bioinformatics scientists.

## Chapter 2 Translating DNA sequences into a protein sequences

After clearning the contents of sequence databases such as GenBank and how their file formats are arranged, it is of interest to a biologist to pull data from these files and databanks and manipulate it.

### The raw genetic code
Using the previously used FASTA file, we can pull a DNA sequence from a record:

In [15]:
#in the case of wanting the information stored in the first record, add "next" before the file is read
#this then stores that record in the variable "first_record"
first_record = next(SeqIO.parse("ls_orchid.fasta", "fasta"))

#then reads out the sequence stores in the record using the ".seq" function
print(first_record.seq)

CGTAACAAGGTTTCCGTAGGTGAACCTGCGGAAGGATCATTGATGAGACCGTGGAATAAACGATCGAGTGAATCCGGAGGACCGGTGTACTCAGCTCACCGGGGGCATTGCTCCCGTGGTGACCCTGATTTGTTGTTGGGCCGCCTCGGGAGCGTCCATGGCGGGTTTGAACCTCTAGCCCGGCGCAGTTTGGGCGCCAAGCCATATGAAAGCATCACCGGCGAATGGCATTGTCTTCCCCAAAACCCGGAGCGGCGGCGTGCTGTCGCGTGCCCAATGAATTTTGATGACTCTCGCAAACGGGAATCTTGGCTCTTTGCATCGGATGGAAGGACGCAGCGAAATGCGATAAGTGGTGTGAATTGCAAGATCCCGTGAACCATCGAGTCTTTTGAACGCAAGTTGCGCCCGAGGCCATCAGGCTAAGGGCACGCCTGCTTGGGCGTCGCGCTTCGTCTCTCTCCTGCCAATGCTTGCCCGGCATACAGCCAGGCCGGCGTGGTGCGGATGTGAAAGATTGGCCCCTTGTGCCTAGGTGCGGCGGGTCCAAGAGCTGGTGTTTTGATGGCCCGGAACCCGGCAAGAGGTGGACGGATGCTGGCAGCAGCTGCCGTGCGAATCCCCCATGTTGTCGTGCTTGTCGGACAGGCAGGAGAACCCTTCCGAACCCCAATGGAGGGCGGTTGACCGCCATTCGGATGTGACCCCAGGTCAGGCGGGGGCACCCGCTGAGTTTACGC


The above example extracts the gene sequence from a fasta record. With this information, a biologist can do more than simply readout genetic sequences. Biopython offers ways of manipulating these types of information in a useful manner.

### Transcription
One way is to turn raw genetic sequences, such as the example above, and convert it into an RNA sequence to be later translated into a protein. In essence, biopython lets a biologist perform transcription, and translation, extremely conviniently.

converting DNA into RNA is simple, and can be done as so:

In [20]:
#in the case of wanting the information stored in the first record, add "next" before the file is read
#this then stores that record in the variable "first_record"
first_record = next(SeqIO.parse("ls_orchid.fasta", "fasta"))

#then stores the sequence from that record into the variable "gene_sequnece",
#gene_sequnce has the raw "GCTA" genetic code of the gene
gene_sequence = first_record.seq

#then use the biopython command "transcribe" to transcribe the sequence into RNA
#the store this information into the variable mRNA_sequence, and display it:
RNA_sequence = gene_sequence.transcribe()
print(mRNA_sequence)

CGUAACAAGGUUUCCGUAGGUGAACCUGCGGAAGGAUCAUUGAUGAGACCGUGGAAUAAACGAUCGAGUGAAUCCGGAGGACCGGUGUACUCAGCUCACCGGGGGCAUUGCUCCCGUGGUGACCCUGAUUUGUUGUUGGGCCGCCUCGGGAGCGUCCAUGGCGGGUUUGAACCUCUAGCCCGGCGCAGUUUGGGCGCCAAGCCAUAUGAAAGCAUCACCGGCGAAUGGCAUUGUCUUCCCCAAAACCCGGAGCGGCGGCGUGCUGUCGCGUGCCCAAUGAAUUUUGAUGACUCUCGCAAACGGGAAUCUUGGCUCUUUGCAUCGGAUGGAAGGACGCAGCGAAAUGCGAUAAGUGGUGUGAAUUGCAAGAUCCCGUGAACCAUCGAGUCUUUUGAACGCAAGUUGCGCCCGAGGCCAUCAGGCUAAGGGCACGCCUGCUUGGGCGUCGCGCUUCGUCUCUCUCCUGCCAAUGCUUGCCCGGCAUACAGCCAGGCCGGCGUGGUGCGGAUGUGAAAGAUUGGCCCCUUGUGCCUAGGUGCGGCGGGUCCAAGAGCUGGUGUUUUGAUGGCCCGGAACCCGGCAAGAGGUGGACGGAUGCUGGCAGCAGCUGCCGUGCGAAUCCCCCAUGUUGUCGUGCUUGUCGGACAGGCAGGAGAACCCUUCCGAACCCCAAUGGAGGGCGGUUGACCGCCAUUCGGAUGUGACCCCAGGUCAGGCGGGGGCACCCGCUGAGUUUACGC


you'll notice now that instead of thymidine (T), the .transcribe() function replaces all thymidines with uracil, effectively making the sequence mRNA.

### Translation
then in regards to translating the sequence. There are a few tools offered to make working with translation relatively simple. Offered by the NCBI are translation codon tables, these being the triletter code for protein sequencing. The NCBI offers several different code tables to facilitate translations from organisms that use different codon tables.

Tables such as these:

In [25]:
#fetching the codon table, denoted as the standard table form the NCBI
#store the table, and display it using the "standard_Code" variable
standard_code = CodonTable.unambiguous_dna_by_name["Standard"]
print(standard_code)

Table 1 Standard, SGC0

  |  T      |  C      |  A      |  G      |
--+---------+---------+---------+---------+--
T | TTT F   | TCT S   | TAT Y   | TGT C   | T
T | TTC F   | TCC S   | TAC Y   | TGC C   | C
T | TTA L   | TCA S   | TAA Stop| TGA Stop| A
T | TTG L(s)| TCG S   | TAG Stop| TGG W   | G
--+---------+---------+---------+---------+--
C | CTT L   | CCT P   | CAT H   | CGT R   | T
C | CTC L   | CCC P   | CAC H   | CGC R   | C
C | CTA L   | CCA P   | CAA Q   | CGA R   | A
C | CTG L(s)| CCG P   | CAG Q   | CGG R   | G
--+---------+---------+---------+---------+--
A | ATT I   | ACT T   | AAT N   | AGT S   | T
A | ATC I   | ACC T   | AAC N   | AGC S   | C
A | ATA I   | ACA T   | AAA K   | AGA R   | A
A | ATG M(s)| ACG T   | AAG K   | AGG R   | G
--+---------+---------+---------+---------+--
G | GTT V   | GCT A   | GAT D   | GGT G   | T
G | GTC V   | GCC A   | GAC D   | GGC G   | C
G | GTA V   | GCA A   | GAA E   | GGA G   | A
G | GTG V   | GCG A   | GAG E   | GGG G   | G
--+---------

Using tables such as these, one can then use commands that can translate RNA sequences.

The code to perform a standard translation of the previously used gene from above is as follows:

In [2]:
#in the case of wanting the information stored in the first record, add "next" before the file is read
#this then stores that record in the variable "first_record"
first_record = next(SeqIO.parse("ls_orchid.fasta", "fasta"))

#then stores the sequence from that record into the variable "gene_sequnece",
#gene_sequnce has the raw "GCTA" genetic code of the gene
gene_sequence = first_record.seq

#then use the biopython command ".transcribe()" to transcribe the sequence into RNA
#the store this information into the variable mRNA_sequence, and display it:
RNA_sequence = gene_sequence.transcribe()

#then use the biopython command ".translate()" and for the sake of keeping track, specify the standard table in the translation
pro_sequence = RNA_sequence.translate(table=1)
print(pro_sequence)

RNKVSVGEPAEGSLMRPWNKRSSESGGPVYSAHRGHCSRGDPDLLLGRLGSVHGGFEPLARRSLGAKPYESITGEWHCLPQNPERRRAVACPMNFDDSRKRESWLFASDGRTQRNAISGVNCKIP*TIESFERKLRPRPSG*GHACLGVALRLSPANACPAYSQAGVVRM*KIGPLCLGAAGPRAGVLMARNPARGGRMLAAAAVRIPHVVVLVGQAGEPFRTPMEGG*PPFGCDPRSGGGTR*VY


C:\Users\Robert\miniconda3\lib\site-packages\Bio\Seq.py:2738: BiopythonWarning: Partial codon, len(sequence) not a multiple of three. Explicitly trim the sequence or add trailing N before translation. This may become an error in future.
  warnings.warn(


However, you should **get an error** if you do try to translate this sequence. Biopython will make note of the fact that the chosen sequence is not a multipule of three, and there may be an issue with matching a codon to some of the remaining nucleotides.

However, it does give a translated sequence with the inputed RNA regardless.

## Chapter 5 Substitution Matrices
One of the fundamental operations in bioinformatics is comparing two sequences, either nucleic acids or proteins, and line them up to archieve maximal levels of identity. This is called _**pairwise sequence alignment**_. Quantification of the similarity of the two sequences is important for establishing whether they are homologs. There are two widely used methods developed to quantify the similarity.

- Method 1: align closely related homologs and count the frequencies of amino acid substitutions.
- Method 2: use a database of aligned sequences derived from protein domains that have a particular structure or function. The frequencies of amino acid substitutions are recorded.

These two methods gave rise to the *PAM* and *BLOSUM* series of amino acid substitution matrices, respectively. These substitution matrices are used in many sequence alignment tools.

The *Biopython* package includes these substitution matrices.

In [ ]:
# this scriptlet display BLOSUM62 and PAM250 matrices
from Bio.Align import substitution_matrices as smatrices
blosum62 = smatrices.load("BLOSUM62")
pam250 = smatrices.load("PAM250")
print(blosum62)
print("-"*80)
print(pam250)

## Chapter 7 Molecular Graphics

In this section, we use NGLView to display a structure of interest.


In [ ]:
#In this block, the user will be prompted to enter the name of the pdb code that they would like to look at.
text_box = str(input("Please enter the PDB code of the structure: "))

In [ ]:
# we minic the folder structure of PDB database and save the pdf file in the corresponding folder.
first_pdb_file = PDBList()
name = first_pdb_file.retrieve_pdb_file(text_box)
protein_file = ""
last_value = -1
print(name[-2])

if (platform.system() == 'Windows'):
    while (name[last_value] != '\\'):
        protein_file += name[last_value] #Each of the letters is added to the protein_name string, starting from the last letter.
        #The value of last_value (which is supposed to immitate the index) is reduced by one, and since it's negative, the constantly decreasing value goes towards the beginning of the string.
        last_value -= 1 

else:
    while (name[last_value] != '/'):
        protein_file += name[last_value] #Each of the letters is added to the protein_name string, starting from the last letter.
        #The value of last_value (which is supposed to immitate the index) is reduced by one, and since it's negative, the constantly decreasing value goes towards the beginning of the string.
        last_value -= 1 

protein_file = protein_file[::-1] #The protein name is reversed, to get a proper pdb file format.

protein_name = ""

index = 0
while(protein_file[index] != '.'):
    protein_name += protein_file[index]
    index += 1

protein_name = protein_name.upper()
print("The name of the protein is, " + protein_file)

In [ ]:
protein_class = str(protein_file[1]) + str(protein_file[2])

#We create an instance of the MMCIF Parser, to load the protein file.
parser = MMCIFParser()

path = os.path.join(protein_class, protein_file)
structure = parser.get_structure(protein_name, path)
path = os.path.join(protein_class, protein_file)
structure = parser.get_structure(protein_name, path)

In [ ]:

def clean_protein(obj):
    print(', '.join([a for a in dir(obj) if not a.startswith('_')]))
    
clean_protein(structure)

In [ ]:
view_one = nv.show_biopython(structure)
view_one

In [ ]:
view_two = nv.show_biopython(structure)
view_two.add_ball_and_stick()
view_two

In [ ]:
# clean up the folder if neccessary
os.remove(path) #Removes the file after the user is done looking at the protein.
os.rmdir(protein_class)